In [1]:
import os, glob
import pandas as pd
from helper import *
import datetime
import numpy as np
pwd = os.path.dirname(os.path.realpath('.'))
data_folder = pwd + '/Data'
data_2018 = data_folder + '/2018'

In [2]:
list_of_speed_files_2018 = glob.glob(data_2018+'/speed_for_chunks/*')
list_of_csv_files_2018 = glob.glob(data_2018+'/2018_results/*')
annotations = pd.read_excel(data_2018+'/annotation.xlsx')
# len(list_of_csv_files_2018)

In [3]:
#  filter unvalid files
annotations = annotations[annotations.is_valid == 'Y']

In [ ]:
name_of_csv_files_2018 = [i.split('/')[-1].split('.')[0] for i in list_of_csv_files_2018]
valid_files = [i for i in annotations.filename]
cnt = 0
for i in valid_files:
    if i in name_of_csv_files_2018:
        cnt += 1
assert cnt == len(valid_files), "Not all the files found!"

In [ ]:
cnt = 0
dataset = pd.DataFrame()
not_processed = []
for j in annotations.filename:
    for i in name_of_csv_files_2018:
        if i == j:
            print("Processing file: ", j)
            try:
                file = [k for k in list_of_speed_files_2018 if k.split('/')[-1].split('.')[0][:-6] == j][0]
            except:
                print('Could not process ', j)
                print('Speed profile could not be found!')
                not_processed.append(j)
                break
            df_raw_speed = prepare_speed_profiles_for_model(file)

            file = [k for k in list_of_csv_files_2018 if k.split('/')[-1].split('.')[0] == j][0]

            df_detection = prepare_detections_for_model(file, df_raw_speed)
            df_detection = merge_speed_and_detection(df_detection, df_raw_speed, annotations, filename=i)

            dataset = pd.concat([dataset,df_detection])

            cnt += 1

    if cnt%10 == 0:
        print(len(list_of_speed_files_2018)-cnt, " files remained!")
        print(cnt, ' file processed!')

Processing file:  unit01_2018-09-26_20_38_20_1_2832
Processing file:  unit01_2018-09-26_22_29_30_1_7032
Processing file:  unit01_2018-09-26_22_29_30_4_28392
Processing file:  unit01_2018-09-26_23_17_50_3_25560
Processing file:  unit01_2018-10-17_15_03_17_15_83064
Processing file:  unit01_2018-10-18_15_22_15_1_66288
Processing file:  unit01_2018-10-19_21_13_40_1_1
Processing file:  unit01_2018-10-20_16_57_49_2_103512
Processing file:  unit01_2018-10-20_16_57_49_3_105384
Processing file:  unit01_2018-10-21_19_16_24_1_480
433  files remained!
10  file processed!
Processing file:  unit01_2018-10-21_19_16_24_2_2520
Processing file:  unit01_2018-10-21_22_59_02_1_1272
Processing file:  unit01_2018-10-21_22_59_02_2_12192
Processing file:  unit01_2018-10-21_22_59_02_4_20376
Processing file:  unit01_2018-10-22_20_55_26_8_257952
Processing file:  unit01_2018-10-24_20_15_45_17_292464
Processing file:  unit01_2018-10-26_17_54_18_2_89472
Processing file:  unit01_2018-10-26_17_54_18_3_90888
Processin

In [ ]:
dataset.to_csv('../Data/dataset.csv')